In [92]:
import torch
from torch import nn
import torchvision
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

In [93]:
writer = SummaryWriter()

In [94]:
fashion_mnist_train = torchvision.datasets.FashionMNIST(root='data', train=True, download=True, transform=ToTensor()) # normalized
fashion_mnist_test = torchvision.datasets.FashionMNIST(root='data', train=False, download=True, transform=ToTensor()) # normalized
len(fashion_mnist_train), len(fashion_mnist_test)

(60000, 10000)

In [95]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.model =  nn.Sequential(nn.Conv2d(1,6,5, padding=2),
                                    nn.Sigmoid(),
                                    nn.AvgPool2d(2, stride=2),
                                    nn.Conv2d(6,16,5),
                                    nn.Sigmoid(),
                                    nn.AvgPool2d(2, stride=2),
                                    nn.Flatten(),
                                    nn.Linear(400, 120),
                                    nn.Sigmoid(),
                                    nn.Linear(120, 84),
                                    nn.Sigmoid(),
                                    nn.Linear(84, 10))

    def forward(self, x):
        x = self.model(x)
        return x

In [96]:
lenet = LeNet()
lenet

LeNet(
  (model): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): Sigmoid()
    (2): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=400, out_features=120, bias=True)
    (8): Sigmoid()
    (9): Linear(in_features=120, out_features=84, bias=True)
    (10): Sigmoid()
    (11): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [97]:
epochs = 50
learning_rate = 0.1
batch_size = 64

In [98]:
dataloader_train = DataLoader(fashion_mnist_train, batch_size=batch_size, shuffle=True)
dataloader_test = DataLoader(fashion_mnist_test, batch_size=batch_size, shuffle=True)

num_train_batches = len(dataloader_train)
num_test_batches = len(dataloader_test)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(lenet.parameters(), lr=learning_rate)

In [99]:
for epoch in range(epochs):
    train_loss = 0
    train_acc = 0

    lenet.train()
    for batch_idx, (train_features, train_labels) in enumerate(dataloader_train):
        
        optimizer.zero_grad()

        predictions = lenet(train_features)
        predictions_labels = torch.argmax(predictions, dim=1)

        train_batch_acc = (predictions_labels == train_labels).sum().item() / train_features.shape[0]

        train_batch_loss = loss(predictions, train_labels)
        train_batch_loss.backward()

        optimizer.step()

        train_loss += train_batch_loss.item()
        train_acc += train_batch_acc

    val_loss = 0
    val_acc = 0

    lenet.eval()
    with torch.no_grad():
        for batch_idx, (test_features, test_labels) in enumerate(dataloader_test):
            predictions = lenet(test_features)
            predictions_labels = torch.argmax(predictions, dim=1)

            test_batch_acc = (predictions_labels == test_labels).sum().item() / test_features.shape[0]
            test_batch_loss = loss(predictions, test_labels)

            val_loss += test_batch_loss.item()
            val_acc += test_batch_acc

    train_loss /= num_train_batches
    train_acc /= num_train_batches

    val_loss /= num_test_batches
    val_acc /= num_test_batches

    writer.add_scalar("Loss/train", train_loss, epoch)
    writer.add_scalar('Accuracy/train', train_acc, epoch)

    writer.add_scalar("Loss/val", val_loss, epoch)
    writer.add_scalar('Accuracy/val', val_acc, epoch)

In [100]:
writer.flush()